## Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Put Data in DataFrame

In [ ]:
import pandas as pd
import glob, os

# Place dataset path here
path = "../Datasets/CoAID/"

dfFalse = pd.read_csv(path+"NewsFakeCOVID-19.csv", usecols=['title'])
dfFalse['label']=0
dfFalseJuly = pd.read_csv(path+"NewsFakeCOVID-19-JULY.csv", usecols=['title'])
dfFalseJuly['label']=0
dfTrue = pd.read_csv(path+"NewsRealCOVID-19.csv", usecols=['title'], nrows=len(dfFalse.values))
dfTrue['label']=1
dfTrueJuly = pd.read_csv(path+"NewsRealCOVID-19-JULY.csv", usecols=['title'], nrows=len(dfFalseJuly.values))
dfTrueJuly['label']=1

dfTotal = pd.concat([dfTrue, dfFalse, dfTrueJuly, dfFalseJuly])

X = dfTotal['title'].values
y = dfTotal['label'].values
print("len(X)", len(X))
print("len(y)", len(y))

len(X) 1676
len(y) 1676


## Finetune BERT

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 8.1 MB/s 
     |████████████████████████████████| 140 kB 91.4 MB/s 
     |████████████████████████████████| 212 kB 84.6 MB/s 
     |████████████████████████████████| 1.1 MB 79.5 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 98.9 MB/s 
     |████████████████████████████████| 127 kB 108.5 MB/s 
     |████████████████████████████████| 271 kB 81.7 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 144 kB 98.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's depe

In [ ]:
from datasets import Dataset
from sklearn.utils import shuffle
dfTotal = shuffle(dfTotal)
train_dataset = Dataset.from_pandas(dfTotal)

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')

train_dataset = train_dataset.map(tokenize_data, batched=True)


from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)


from transformers import Trainer

trainer_train = Trainer(
    model=model, args=training_args, train_dataset=train_dataset
)

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Parameter 'function'=<function tokenize_data at 0x7f6e9a72d050> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
trainer_train.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1676
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 630


Step,Training Loss
500,0.082300


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=630, training_loss=0.0671632484784202, metrics={'train_runtime': 284.9931, 'train_samples_per_second': 17.643, 'train_steps_per_second': 2.211, 'total_flos': 1322922386350080.0, 'train_loss': 0.0671632484784202, 'epoch': 3.0})

In [ ]:
import torch
model_path = '../models/roberta-fake-news_CoAID.pt'
torch.save(model.state_dict(),model_path)


In [ ]:
path_to_external_validation_dataset = ".../Datasets/External_Validation_Datasets//" # Replace with appropriate path
dfTotal_external = pd.read_csv(path_to_external_validation_dataset+"External_Validation1.csv", usecols=['title', 'label'])

def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0
dfTotal_external = dfTotal_external.dropna()
dfTotal_external['label'] = dfTotal_external['label'].apply(lambda label: binaryLabel(str(label)))
dataset_external = Dataset.from_pandas(dfTotal_external)

In [ ]:
import numpy as np
from datasets import load_metric
from sklearn.metrics import precision_recall_fscore_support,accuracy_score

metric = load_metric("accuracy")


def compute_metrics(eval_pred)):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

dataset_external = dataset_external.map(tokenize_data, batched=True)

trainer_eval = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
trainer_eval.evaluate()

  0%|          | 0/8 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.9562136885362051,
 'eval_loss': 0.2911560833454132,
 'eval_runtime': 128.1484,
 'eval_samples_per_second': 55.069,
 'eval_steps_per_second': 6.89}

# External Validation2

In [ ]:
dfTotal_external = pd.read_csv(path_to_external_validation_dataset = ".../Datasets/External_Validation_Datasets//" # Replace with appropriate path
dfTotal_external = pd.read_csv(path_to_external_validation_dataset+"External_Validation2.csv", usecols=['title', 'label'])
)
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

trainer_eval = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
trainer_eval.evaluate()

  0%|          | 0/31 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30630
  Batch size = 8


{'eval_accuracy': 0.8097616715638263,
 'eval_loss': 1.8956300020217896,
 'eval_runtime': 555.4705,
 'eval_samples_per_second': 55.142,
 'eval_steps_per_second': 6.893}